In [316]:
import xmltodict
import pandas as pd
import os
import re

In [110]:
def listRecursive(d, key, path = None):
    if not path: path = []
    for k, v in d.items ():
        if isinstance (v, dict):
            for path, found in listRecursive(v, key, path + [k]):
                yield path, found
        elif isinstance(v, list):
            for i, d in enumerate(v):
                for path, found in listRecursive(d, key, path + [i]):
                    yield path, found
        if k == key:
            yield path+[k], v


example = {'app_url': '', 'models': [{'perms': {'add': True, 'change': True, 'delete': True}, 'add_url': '/admin/cms/news/add/', 'admin_url': '/admin/cms/news/', 'name': ''}], 'has_module_perms': True, 'name': u'CMS'}

for path, found in listRecursive (example, 'admin_url'):
    print (path, found)

[0, 'admin_url'] /admin/cms/news/


In [181]:
def remove_tags(read_file):
    read_file = re.sub(b'</?external-xref[^<>]*>',b'', read_file)
    read_file = re.sub(b'<quote>',b'"', read_file)
    read_file = re.sub(b'</quote>',b'"', read_file)
    read_file = re.sub(b'</?term[^<>]*>',b' ', read_file)
    read_file = re.sub(b'</?pagebreak[^<>]*>',b'', read_file)
    return read_file

In [424]:
def flatten(d):
    output = ''
    for k, v in d.items():
        if not k.startswith("@"):
            if isinstance(v, dict):
                output += flatten(v)
            elif isinstance(v, list):
                for l in v:
                    if isinstance(l, dict):
                        output += flatten(l)
                    else:
                        if l:
                            output += l + ' '
            else:
                if v:
                    output += v + ' '
    return output

In [425]:
# organizing the original xml data: 
# 1) Add file-name: just file name
# 2) Add bill-type: there are three types: 'bill', 'resolution', 'amendment-doc'
# 3) Grouped all "meta" data which start with @ from parser.
def bill_to_dict(filename, doc):
    def _clean_body(bodydict):
        clean_bodydict = {}
        for k, v in bodydict.items():
            if not k.startswith("@"):
                if isinstance(v, list):
                    sec_str = ''
                    for sub_dict in v:
                        if sub_dict:
                            sec_str += flatten(sub_dict)
                    clean_bodydict[k] = sec_str
#                 elif isinstance(v, dict):
                else:
                    clean_bodydict[k] = flatten(v)
#                 else:
#                     print('xxxxxxxxx')
        return clean_bodydict
    
    bill_type = list(doc)[0]
    data_dict = {}
    
    data_dict["file-name"] = filename
    data_dict["bill-type"] = bill_type
    
    metadata = {}
    for n in list(doc[bill_type]):
        # group meta data
        if n.startswith("@"): 
            metadata[n] = doc[bill_type][n]
        # unify name for different types
        elif n in ['legis-body', 'resolution-body', 'engrossed-amendment-body']:
            if isinstance(doc[bill_type][n], dict):
                data_dict['body'] = _clean_body(doc[bill_type][n])
            elif isinstance(doc[bill_type][n], list):
                # just take last one
                data_dict['body'] = _clean_body(doc[bill_type][n][-1])
            else:
                print('NOT dict nor list')
                data_dict['body'] = doc[bill_type][n]
                
        elif n == 'engrossed-amendment-form':
            data_dict['form'] = doc[bill_type][n]
        else:
            data_dict[n] = doc[bill_type][n]
    data_dict["metadata"] = metadata
    return data_dict

In [361]:
with open('bill_text_115/92510.140382952224512.47152.xml', 'rb') as f:
    test_doc = xmltodict.parse(remove_tags(f.read()))

In [374]:
bill_to_dict('92510.140382952224512.47152.xml', test_doc)['body']['section']

'1. Permanent resident status for Joseph\n                Gabra and Sharon Kamel (a) In\n                    general Notwithstanding subsections (a) and (b) of section 201 of the Immigration and Nationality Act\n                    (8 U.S.C. 1151), Joseph\n                    Gabra and Sharon Kamel shall each be eligible for issuance of an immigrant visa or for adjustment of\n                    status to that of an alien lawfully admitted for permanent residence upon filing an application for\n                    issuance of an immigrant visa under section 204 of such Act (\n                        8 U.S.C. 1154) or for adjustment of status to lawful permanent resident. (b) Adjustment of\n                    status Immigration and Nationality Act If Joseph Gabra or Sharon Kamel enters the United States before the filing deadline specified in\n                    subsection (c), Joseph Gabra or Sharon Kamel shall be considered to have entered and remained\n                    lawfully 

In [426]:
# I put all the files in one folder for simplicity
indir = 'bill_text_115' 

data = [] # a list of dict, each element is a xml file

for root, dirs, filenames in os.walk(indir):
    for filename in filenames:
        with open(os.path.join(indir, filename), 'rb') as f:
            doc = xmltodict.parse(remove_tags(f.read()))
            data.append(bill_to_dict(filename, doc))

In [427]:
[list(x) for x in set(tuple(x) for x in data)]

[['metadata', 'bill-type', 'preamble', 'body', 'file-name', 'form'],
 ['metadata', 'body', 'file-name', 'bill-type', 'form'],
 ['metadata',
  'bill-type',
  'body',
  'file-name',
  'form',
  'official-title-amendment',
  'endorsement'],
 ['metadata',
  'bill-type',
  'title-amends',
  'body',
  'file-name',
  'form',
  'endorsement',
  'attestation'],
 ['metadata',
  'bill-type',
  'preamble',
  'body',
  'file-name',
  'form',
  'endorsement',
  'attestation'],
 ['metadata',
  'bill-type',
  'preamble',
  'body',
  'file-name',
  'form',
  'endorsement'],
 ['metadata',
  'bill-type',
  'body',
  'file-name',
  'form',
  'endorsement',
  'attestation'],
 ['metadata', 'bill-type', 'body', 'file-name', 'form', 'endorsement'],
 ['metadata',
  'bill-type',
  'preamble',
  'body',
  'file-name',
  'form',
  'attestation'],
 ['metadata', 'bill-type', 'body', 'file-name', 'form', 'attestation']]

In [432]:
select_keys = ['metadata', 'bill-type', 'body', 'file-name', 'form']
select_data = []
for d in data:
    select_data.append({ select_key: d[select_key] for select_key in select_keys })

In [420]:
df = pd.io.json.json_normalize(select_data)

In [421]:
df

,bill-type,body.amendment,body.division,body.section,body.title,file-name,metadata.@amend-degree,metadata.@amend-stage,metadata.@amend-type,metadata.@bill-stage,metadata.@bill-type,metadata.@dms-id,metadata.@key,metadata.@public-print,metadata.@public-private,metadata.@resolution-stage,metadata.@resolution-type,metadata.@stage-count,metadata.@star-print
0,bill,NaN,NaN,1. Permanent resident status for Joseph\n ...,NaN,92510.140382952224512.47152.xml,NaN,NaN,NaN,Introduced-in-Senate,NaN,A1,NaN,NaN,private,NaN,NaN,NaN,NaN
1,resolution,NaN,NaN,None That it is the sense of Congress that— (1...,NaN,US_Bill_Text_115_HCONRES10_IH.xml,NaN,NaN,NaN,NaN,NaN,H6105DDE2819B44DBB5E467A21768B92E,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
2,resolution,NaN,NaN,None That it is the sense of Congress that— (1...,NaN,US_Bill_Text_115_HCONRES11_IH.xml,NaN,NaN,NaN,NaN,NaN,H4AC6CF641CD640E393CFA510D0960D8C,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
3,resolution,NaN,NaN,None That Congress— (1) supports the designati...,NaN,US_Bill_Text_115_HCONRES12_IH.xml,NaN,NaN,NaN,NaN,NaN,H4E5CA555E82D40E08E6EDAC6DB9540D3,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
4,resolution,NaN,NaN,None That Congress should not impose any new p...,NaN,US_Bill_Text_115_HCONRES13_IH.xml,NaN,NaN,NaN,NaN,NaN,HE39F52BA0A584A39A31BF533005200CC,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
5,resolution,NaN,NaN,1. Short title This resolution may be cited as...,NaN,US_Bill_Text_115_HCONRES14_IH.xml,NaN,NaN,NaN,NaN,NaN,H40B69ED168D842FCA4473EBF04679DB6,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
6,resolution,NaN,NaN,None That— (1) Donald J. Trump won the 2016 pr...,NaN,US_Bill_Text_115_HCONRES15_IH.xml,NaN,NaN,NaN,NaN,NaN,HEC124C4E28034F0C8032E3BCCB955FBD,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
7,resolution,NaN,NaN,None That it is the sense of Congress that— (1...,NaN,US_Bill_Text_115_HCONRES16_IH.xml,NaN,NaN,NaN,NaN,NaN,HCC74C220526D4AFDAF81C0E2148FD2CB,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
8,resolution,NaN,NaN,None That— 1. Short title This resolution may ...,NaN,US_Bill_Text_115_HCONRES17_IH.xml,NaN,NaN,NaN,NaN,NaN,H2AC8F7FC62CD410EA708DFCD97016B45,H,NaN,public,Introduced-in-House,house-concurrent,NaN,no-star-print
9,resolution,NaN,NaN,1. Use of rotunda for holocaust days of rememb...,NaN,US_Bill_Text_115_HCONRES18_EH.xml,NaN,NaN,NaN,NaN,NaN,H926C8CB44FFE4D25B18DD87483D485A5,H,NaN,public,Engrossed-in-House,house-concurrent,1,no-star-print


### Tried two ways to convert to dataframe
#### 1. pandas.io.json.json_normalize
- This will flat the nested structure. But since some tags have grand and grand grand sons, and some bills has additional information, the column number is large, so the head shows here are NA values.

In [5]:
df = pd.io.json.json_normalize(data)
df.head(2)

,attestation.attestation-group,attestation.attestation-group.attestation-date,attestation.attestation-group.attestation-date.#text,attestation.attestation-group.attestation-date.@chamber,attestation.attestation-group.attestation-date.@date,attestation.attestation-group.attestation-date.@legis-day,attestation.attestation-group.attestor,attestation.attestation-group.attestor.#text,attestation.attestation-group.attestor.@display,attestation.attestation-group.proxy.#text,...,resolution-body.section.text.external-xref.@parsable-cite,resolution-body.section.text.pagebreak,resolution-body.section.text.quote,resolution-body.section.text.quote.#text,resolution-body.section.text.quote.pagebreak,resolution-body.section.text.quote.quote,resolution-body.title,title-amends.official-title-amendment.#text,title-amends.official-title-amendment.pagebreak,title-amends.official-title-amendment.quote
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
list(df)

['attestation.attestation-group',
 'attestation.attestation-group.attestation-date',
 'attestation.attestation-group.attestation-date.#text',
 'attestation.attestation-group.attestation-date.@chamber',
 'attestation.attestation-group.attestation-date.@date',
 'attestation.attestation-group.attestation-date.@legis-day',
 'attestation.attestation-group.attestor',
 'attestation.attestation-group.attestor.#text',
 'attestation.attestation-group.attestor.@display',
 'attestation.attestation-group.proxy.#text',
 'attestation.attestation-group.proxy.@display',
 'attestation.attestation-group.role',
 'bill-type',
 'endorsement',
 'endorsement.@display',
 'endorsement.action-date',
 'endorsement.action-date.#text',
 'endorsement.action-date.@date',
 'endorsement.action-date.@display',
 'endorsement.action-desc',
 'endorsement.action-desc.#text',
 'endorsement.action-desc.@blank-lines-after',
 'endorsement.action-desc.@display',
 'endorsement.action-desc.external-xref.#text',
 'endorsement.actio

In [7]:
omit_cols = ['@display', 
'@blank-lines-after', 
'@display-enacting-clause', 
'@display-resolving-clause',
'@display-inline', 
'@by-request', 
'pagebreak', 
'@reported-display-style', 
'@style']
# 'commented'

In [14]:
cols = list(df)

for i, c in enumerate(cols):
    for oc in omit_cols:
        if oc in c:
            cols.pop(i)

In [15]:
len(cols)

303

In [16]:
len(list(df))

405

In [19]:
cols

['attestation.attestation-group',
 'attestation.attestation-group.attestation-date',
 'attestation.attestation-group.attestation-date.#text',
 'attestation.attestation-group.attestation-date.@chamber',
 'attestation.attestation-group.attestation-date.@date',
 'attestation.attestation-group.attestation-date.@legis-day',
 'attestation.attestation-group.attestor',
 'attestation.attestation-group.attestor.#text',
 'attestation.attestation-group.proxy.#text',
 'attestation.attestation-group.role',
 'bill-type',
 'endorsement',
 'endorsement.action-date',
 'endorsement.action-date.#text',
 'endorsement.action-date.@date',
 'endorsement.action-desc',
 'endorsement.action-desc.#text',
 'endorsement.action-desc.@display',
 'endorsement.action-desc.external-xref.#text',
 'endorsement.action-desc.external-xref.@legal-doc',
 'endorsement.action-desc.external-xref.@parsable-cite',
 'engrossed-amendment-body.amendment',
 'engrossed-amendment-body.amendment.amendment-block',
 'engrossed-amendment-bod

In [29]:
df['legis-body'][df['legis-body'].notnull()]

2697    [{'@changed': 'added', '@committee-id': 'HSM00...
3998    [{'@id': 'H8323BEA593F940BDB963F0FB7F0C90F4', ...
4114    [{'@id': 'HA66C6B057D414BA188079F3D5A8A7A9F', ...
4212    [{'@id': 'H27ED6AC8946C402D827B6329E9A26E5C', ...
4219    [{'@id': 'H7F1917DE5E944E7EAABC94ED10FC4E88', ...
4248    [{'@id': 'HA2B585162BF041279511C83D5F26F9AB', ...
4763    [{'@id': 'H88251086F90A4B4684783ECEC1CC2F33', ...
5546    [{'@display-enacting-clause': 'yes-display-ena...
5555    [{'@display-enacting-clause': 'yes-display-ena...
5565    [{'section': [OrderedDict([('@changed', 'delet...
5589    [{'section': [OrderedDict([('@changed', 'delet...
5638    [{'section': {'@changed': 'deleted', '@id': 'S...
5641    [{'section': [OrderedDict([('@changed', 'delet...
5685    [{'section': [OrderedDict([('@changed', 'delet...
5911    [{'section': [OrderedDict([('@changed', 'delet...
5915    [{'section': [OrderedDict([('@changed', 'delet...
5924    [{'section': [OrderedDict([('@changed', 'delet...
5953    [{'sec

In [ ]:
# df.to_csv("bills.csv")

#### 2. pandas.DataFrame
- Easier to see information like how many bill-type

In [33]:
df2 = pd.DataFrame(data)
df2.head(2)

,attestation,bill-type,endorsement,engrossed-amendment-body,engrossed-amendment-form,file-name,form,legis-body,metadata,official-title-amendment,preamble,resolution-body,title-amends
0,NaN,bill,NaN,NaN,NaN,92510.140382952224512.47152.xml,"{'distribution-code': {'@display': 'yes', '#te...",{'@display-enacting-clause': 'yes-display-enac...,"{'@dms-id': 'A1', '@bill-stage': 'Introduced-i...",NaN,NaN,NaN,NaN
1,NaN,resolution,NaN,NaN,NaN,US_Bill_Text_115_HCONRES10_IH.xml,"{'distribution-code': {'@display': 'yes', '#te...",NaN,"{'@resolution-type': 'house-concurrent', '@pub...",NaN,{'whereas': [{'text': 'Whereas America’s frate...,"{'@id': 'H87EB0F92716248E78751D13F08B91455', '...",NaN


In [34]:
df2['bill-type'].value_counts()

bill             6712
resolution       1312
amendment-doc      15
Name: bill-type, dtype: int64